In [0]:
import numpy as np
from google.colab import files
import pandas as pd
uploaded = files.upload()

Saving Housing.csv to Housing.csv


In [0]:
import io
data = pd.read_csv(io.BytesIO(uploaded['IRIS.csv']))

KeyError: ignored

In [0]:
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
y = data.drop('variety', axis=1)
x = data.variety.astype('category').cat.codes
y = y.rename(columns=y.iloc[0]).drop(y.index[0])
y = y.to_numpy()
x = x.to_numpy()
x = np.delete(x, 0)
y = y.reshape(149, 1, 4)
x = x.reshape(149, 1, 1)
x = np_utils.to_categorical(x)

NameError: ignored

In [0]:
class FCLayer:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = np.random.randn(input_size, output_size) / np.sqrt(input_size + output_size)
        self.bias = np.random.randn(1, output_size) / np.sqrt(input_size + output_size)

    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.bias

    def backward(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # bias_error = output_error
        
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [0]:
class ActivationLayer:
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
    
    def forward(self, input):
        self.input = input
        return self.activation(input)
    
    def backward(self, output_error, learning_rate):
        return output_error * self.activation_prime(self.input)

In [0]:
# bonus
class SoftmaxLayer:
    def __init__(self, input_size):
        self.input_size = input_size
    
    def forward(self, input):
        self.input = input
        tmp = np.exp(input)
        self.output = tmp / np.sum(tmp)
        return self.output
    
    def backward(self, output_error, learning_rate):
        input_error = np.zeros(output_error.shape)
        out = np.tile(self.output.T, self.input_size)
        return self.output * np.dot(output_error, np.identity(self.input_size) - out)

In [0]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return np.exp(-x) / (1 + np.exp(-x))**2

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1 - np.tanh(x)**2

def relu(x):
    return np.maximum(x, 0)

def relu_prime(x):
    return np.array(x >= 0).astype('int')

In [0]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_pred.size

def sse(y_true, y_pred):
    return 0.5 * np.sum(np.power(y_true - y_pred, 2))

def sse_prime(y_true, y_pred):
    return y_pred - y_true

(1, 4)

In [0]:

network = [
    FCLayer(4, 16),
    ActivationLayer(sigmoid, sigmoid_prime),
    FCLayer(16, 3),
    ActivationLayer(sigmoid, sigmoid_prime)

]

epochs = 1000
learning_rate = 0.01

# training
for epoch in range(epochs):
    error = 0
    for xi, y_true in zip(y, x):
        # forward
        output = xi
        for layer in network:
            output = layer.forward(output)
        
        # error (display purpose only)
        error += mse(y_true, output)

        # backward
        output_error = mse_prime(y_true, output)
        for layer in reversed(network):
            output_error = layer.backward(output_error, learning_rate)
    
    error /= len(y)
    print('%d/%d, error=%f' % (epoch + 1, epochs, error))

1/1000, error=0.240303
2/1000, error=0.228309
3/1000, error=0.223345
4/1000, error=0.220862
5/1000, error=0.219277
6/1000, error=0.218020
7/1000, error=0.216874
8/1000, error=0.215749
9/1000, error=0.214606
10/1000, error=0.213424
11/1000, error=0.212190
12/1000, error=0.210892
13/1000, error=0.209522
14/1000, error=0.208070
15/1000, error=0.206529
16/1000, error=0.204895
17/1000, error=0.203166
18/1000, error=0.201346
19/1000, error=0.199441
20/1000, error=0.197460
21/1000, error=0.195412
22/1000, error=0.193303
23/1000, error=0.191137
24/1000, error=0.188917
25/1000, error=0.186647
26/1000, error=0.184330
27/1000, error=0.181972
28/1000, error=0.179578
29/1000, error=0.177158
30/1000, error=0.174722
31/1000, error=0.172279
32/1000, error=0.169842
33/1000, error=0.167421
34/1000, error=0.165026
35/1000, error=0.162669
36/1000, error=0.160358
37/1000, error=0.158100
38/1000, error=0.155902
39/1000, error=0.153770
40/1000, error=0.151707
41/1000, error=0.149716
42/1000, error=0.147799
4

In [0]:
def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

In [0]:
predict(network, [[4.8,3,1.4,0.1]])

array([[9.64966582e-01, 6.61861080e-02, 9.52410064e-05]])